In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# set parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# network parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
keep_prob = tf.placeholder("float", None)

In [3]:
from functools import reduce
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()

# create some wrappers for simplicity
def conv(bottom, num_filter, ksize=3, stride=1, padding="SAME", scope=None):
    
    bottom_shape = bottom.get_shape().as_list()[3]
    
    with tf.variable_scope(scope or "conv"):
        W = tf.get_variable("W", [ksize, ksize, bottom_shape, num_filter], 
                            initializer=he_init)
        b = tf.get_variable("b", [num_filter],
                           initializer=tf.constant_initializer(0))
        
        x = tf.nn.conv2d(bottom, W, 
                         strides=[1, stride, stride, 1], 
                         padding=padding)
        x = tf.nn.relu(tf.nn.bias_add(x, b))
    
    return x


def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope=None):
    
    with tf.variable_scope(scope or "maxpool"):
        pool = tf.nn.max_pool(bottom, ksize=[1, ksize, ksize, 1], 
                              strides=[1, stride, stride, 1],
                              padding=padding)
    return pool


def fc(bottom, num_dims, scope=None):
    
    bottom_shape = bottom.get_shape().as_list()
    if len(bottom_shape) > 2:
        bottom = tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
        bottom_shape = bottom.get_shape().as_list()
        
    with tf.variable_scope(scope or "fc"):
        W = tf.get_variable("W", [bottom_shape[1], num_dims], 
                            initializer=he_init)
        b = tf.get_variable("b", [num_dims],
                           initializer=tf.constant_initializer(0))
        
        out = tf.nn.bias_add(tf.matmul(bottom, W), b)
    return out


def fc_relu(bottom, num_dims, scope=None):
    
    with tf.variable_scope(scope or "fc"):
        out = fc(bottom, num_dims, scope="fc")
        relu = tf.nn.relu(out)
        
    return relu
    

def conv_net(x, keep_prob):
    # reshape input image (784 -> 28x28)
    # Note that input of conv layers are [N, H, W, C] shape
    # in MNIST data, it's gray-scale channel is 1, but in normal case c must be 3 (RGB)
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = conv(x, 32, 5, scope="conv_1")
    conv1 = maxpool(conv1, scope="maxpool_1")
    conv2 = conv(conv1, 64, 5, scope="conv_2")
    conv2 = maxpool(conv2, scope="maxpool_2")
    
    fc1 = fc_relu(conv2, 1024, scope="fc_1")
    fc1 = tf.nn.dropout(fc1, keep_prob)

    out = fc(fc1, n_classes, scope="out")
    return out

In [4]:
pred = conv_net(x, keep_prob)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [7]:
# session config
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

sess = tf.Session(config=config)
sess.run(init)

for epoch in range(training_epochs):
    avg_loss = 0
    total_batch = int(mnist.train.num_examples/batch_size)

    for i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, l = sess.run([optimizer, loss], feed_dict={x: batch_x,
                                                      y: batch_y,
                                                      keep_prob: 0.5})

        avg_loss += l / total_batch
    if epoch % display_step == 0:
        print("Epoch: {0:04d} loss={1:.5f}" .format(epoch+1, avg_loss))
print("Optimization Finished!")

# test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print("Test accuracy: {0}" 
      .format(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})))

Epoch: 0001 loss=0.18113
Epoch: 0002 loss=0.04833
Epoch: 0003 loss=0.03412
Epoch: 0004 loss=0.02707
Epoch: 0005 loss=0.02128
Epoch: 0006 loss=0.01843
Epoch: 0007 loss=0.01380
Epoch: 0008 loss=0.01307
Epoch: 0009 loss=0.01203
Epoch: 0010 loss=0.01418
Epoch: 0011 loss=0.00827
Epoch: 0012 loss=0.00909
Epoch: 0013 loss=0.00876
Epoch: 0014 loss=0.00674
Epoch: 0015 loss=0.00824
Optimization Finished!
Test accuracy: 0.9917001724243164
